In [10]:
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("../RD-systems-and-test-benches/utils")
import model_fsolve as modf
import model_fsolve_v2 as modf2

import scipy.optimize as sco
import scipy.interpolate as sci

import hx_hydraulic as hxhy
import from_excel as fe
import data_plots as dplt

import fluids as fds
from CoolProp.CoolProp import PropsSI

import openpyxl as opxl
from sklearn.ensemble import RandomForestRegressor

import seaborn as sns
import os
import plotly.graph_objects as go

import utils.data_processing as dp
import utils.conversion as conv
import copy

from plot_config import *

In [2]:
path = r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Inputs"
file_name = "\V4.5_wo_interMPE.xlsx"

fluid = {'name':'MPG','glycol_rate':0.4}
temperature=conv.to_K(25)

hw, par, cond = fe.initialize(path, file_name)
fe.change_fluid(cond, fluid)
fe.change_temperature(cond, temperature)

# tabl, res, PL, residuals = modf.PL_fsolve(par, cond) 

In [3]:
# df, PL, df_PL, residuals = modf.PL_fsolve(par, cond)

### Series for one flow rate but changing geometry

In [4]:
D_man_range = np.array([2.2, 1.8, 1.4,1.]) * par['D_man']

In [ ]:
N_panels = 6

Vdot = 600
Qtot = conv.convert_flow_rate(Vdot, 'L/h', 'm3/s')

res_dict = {}

testings_path_list = [os.path.join(r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Thesis", file_name[1:-5] + f"change_D_man_{int(D_man*1000)}.xlsx") for D_man in D_man_range]

for i, D_man in enumerate(D_man_range):
    print('i ', i)

    # Change diameter

    hw, par, cond = fe.initialize(path, file_name)
    fe.change_fluid(cond, fluid)
    fe.change_temperature(cond, temperature)
    hw.change_man_diameter(D_man)
    par = hw.make_dict()

    # Generate abaque

    list_Qin = np.linspace(0.05, 0.95, 7)*Qtot
    list_alpha = np.linspace(0.05, 0.95, 7)

    df_testings = fe.testing_series_Qmax(Qtot, list_Qin, list_alpha, par,cond)
    testings_path = testings_path_list[i]
    df_testings.to_excel(testings_path, index=False)

    # Run abaque for flow rate Vdot (L/h)

    testings_path = testings_path_list[i]

    df_testings = pd.read_excel(testings_path)
    DPin, DPout, DPx = modf2.transfer_func(df_testings)
    tabl, residuals = modf2.PL_fsolve_MPE(N_panels , conv.convert_flow_rate(Vdot, 'L/h', 'm3/s'), DPin, DPout, DPx)

    res_dict[D_man] = {
                        'N_panels' : N_panels,
                        'Vdot' : Vdot,
                        'tabl' : copy.deepcopy(tabl),
                        'residuals' : copy.deepcopy(residuals)
                        }

    print('---')

In [ ]:
for key in res_dict.keys():
    # key = list(res_dict.keys())[1]
    tabl = res_dict[key]['tabl']
    plt.scatter(tabl.index, tabl['qx']/(tabl['qx'].sum()/N_panels), label=f'D_man = {key*1000} mm')
    print('D_man', int(key*1000), ' mm ', tabl['Pin'][0])

plt.legend()

In [12]:
hw, par, cond = fe.initialize(path, file_name)

option = 'MANIFOLD' + '_'

for (letter, res_dict, x_range) in zip(['man'], [res_dict], [D_man_range]):

    res_dict['parameters'] = {
                                'par' : copy.deepcopy(par),
                                'cond' : copy.deepcopy(cond),
                                'N_panels' : N_panels,
                                'Vdot' : Vdot,
                                'Qtot' : Qtot,
                                'x_range' : x_range
                                }

    dp.write_pickle(r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Thesis", f"V4.5_multipanels_parametric_{option}res_dict_{letter}", res_dict)